# Imports

In [44]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from yellowbrick.classifier import ConfusionMatrix

# Loading data

In [22]:
data = '/home/uadson/repos/machine-learn/data/database/chatlocaldb.sqlite3'

In [23]:
conn = sqlite3.connect(data)

department_qs = "SELECT * FROM departments_department;"
services_qs = "SELECT * FROM services_service;"
questions_qs = "SELECT * FROM questions_question;"

department_df = pd.read_sql_query(department_qs, conn)
services_df = pd.read_sql_query(services_qs, conn)
questions_df = pd.read_sql_query(questions_qs, conn)

# Preprocessing

In [24]:
department_df.head()

,id,created_at,updated_at,active,name,category
0,1,2023-09-13 14:37:17,2023-09-13 14:37:28.602000,1,Secretaria Municipal De Finanças,Finanças


In [25]:
services_df.head()

,id,created_at,updated_at,active,name,index,department_id,profile_id
0,1,2023-09-15 15:04:26.549000,2024-01-16 13:26:17.796000,1,ENTENDA SEU IPTU,1,1,4
1,2,2023-09-15 15:14:38.771000,2023-09-15 15:14:38.775000,1,NOTA FISCAL,2,1,5
2,3,2023-09-15 15:32:25.555000,2024-01-24 10:06:59.069000,1,SIMPLES NACIONAL 2024,3,1,5
3,4,2023-09-15 15:42:50.880000,2023-09-18 13:36:32.251000,1,CADASTRO MOBILIÁRIO 1,4,1,5
4,5,2023-09-15 15:47:02.408000,2023-09-15 16:12:51.991000,1,ISS,5,1,5


In [26]:
questions_df.head()

,id,created_at,updated_at,active,title,answer,url,index,department_id,profile_id,service_id
0,1,2023-09-15 15:09:14.343000,2024-01-16 15:33:35.386000,1,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,https://www.goiania.go.gov.br/sefin/iptu/,1,1,4,1
1,2,2023-09-15 15:11:04.308000,2023-12-05 11:04:00.894000,1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,None,2,1,4,1
2,3,2023-09-15 15:12:06.972000,2024-01-16 14:00:26.230000,1,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",None,3,1,4,1
3,4,2023-09-15 15:12:54.616000,2024-01-16 14:02:04.196000,1,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",None,4,1,4,1
4,5,2023-09-15 15:15:50.990000,2023-12-04 13:17:34.529000,1,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,None,1,1,5,2


In [27]:
questions = questions_df["title answer department_id service_id".split(" ")]
questions.head()

,title,answer,department_id,service_id
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,1,1
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,1,1
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",1,1
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",1,1
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,1,2


In [28]:
questions.columns = ["title answer department service".split(" ")]
questions.head()

,title,answer,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,1,1
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,1,1
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",1,1
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",1,1
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,1,2


In [29]:
questions["department"] = questions["department"].replace(1, "Finanças")
questions.head()

/tmp/ipykernel_910844/2846663326.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["department"] = questions["department"].replace(1, "Finanças")


,title,answer,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,Finanças,1
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,Finanças,1
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",Finanças,1
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",Finanças,1
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,Finanças,2


In [30]:
for question_col in range(len(questions["service"])):
    question_service_pk = questions.iloc[question_col, 3] # id da coluna service
    for service_col in range(len(services_df["id"])):
        service_pk = services_df.iloc[service_col, 0] # id da coluna id em service_df
        
        if question_service_pk == service_pk:
            questions["service"] = questions["service"].replace(question_service_pk, services_df.iloc[service_col, 4])

questions.head()

/tmp/ipykernel_910844/3801845715.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions["service"] = questions["service"].replace(question_service_pk, services_df.iloc[service_col, 4])


,title,answer,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,Finanças,ENTENDA SEU IPTU
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,Finanças,ENTENDA SEU IPTU
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",Finanças,ENTENDA SEU IPTU
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",Finanças,ENTENDA SEU IPTU
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,Finanças,NOTA FISCAL


In [46]:
X_questions = questions["department service title".split(" ")]

X_questions

,department,service,title
0,Finanças,ENTENDA SEU IPTU,Como emitir o boleto do IPTU 2024?
1,Finanças,ENTENDA SEU IPTU,Onde encontro o número de inscrição do meu imó...
2,Finanças,ENTENDA SEU IPTU,Qual será o reajuste do IPTU em 2024?
3,Finanças,ENTENDA SEU IPTU,Quem tem direito à isenção do IPTU 2024?
4,Finanças,NOTA FISCAL,Como faço para cancelar a Nota Fiscal?
...,...,...,...
129,Finanças,SIMPLES NACIONAL 2024,Quem deve solicitar opção pelo Simples Nacional?
130,Finanças,SIMPLES NACIONAL 2024,Qual o prazo para solicitar opção?
131,Finanças,SIMPLES NACIONAL 2024,"Uma vez solicitada, a opção pelo Simples Nacio..."
132,Finanças,SIMPLES NACIONAL 2024,"Após cancelada, poderá ser formalizada nova op..."


In [47]:
y_questions = questions["answer"]

y_questions

,answer
0,O procedimento é simples. Após acessar o porta...
1,Em qualquer boleto de IPTU dos anos anteriores...
2,"Até o ano de 2025, o IPTU não terá reajuste do..."
3,"Em Goiânia, o Código Tributário Municipal busc..."
4,O próprio contribuinte pode efetuar o cancelam...
...,...
129,Os contribuintes que foram excluídos por qualq...
130,A opção deverá ser formalizada até o último di...
131,Enquanto não vencido o prazo para formalização...
132,"Após cancelado o pedido, poderá ser formalizad..."


In [33]:
y_questions = questions.iloc[:, 3]

In [43]:
y_questions

0                                       ENTENDA SEU IPTU
1                                       ENTENDA SEU IPTU
2                                       ENTENDA SEU IPTU
3                                       ENTENDA SEU IPTU
4                                           NOTA FISCAL 
                             ...                        
129                                SIMPLES NACIONAL 2024
130                                SIMPLES NACIONAL 2024
131                                SIMPLES NACIONAL 2024
132                                SIMPLES NACIONAL 2024
133    PERGUNTAS E RESPOSTAS SOBRE O SIMPLES NACIONAL...
Name: (service,), Length: 134, dtype: object